# Where to begin...

In [ ]:
# Setup the environment

%pip install -qU google-genai==1.29.0 lmnr[all]

import os
from google import genai
from google.genai import types
from lmnr import Laminar

class UserSecretsClient:
    @classmethod
    def get_secret(cls, id: str):
        try:
            return os.environ[id]
        except KeyError as e:
            print(f"KeyError: authentication token for {id} is undefined")

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

client = genai.Client(api_key=GOOGLE_API_KEY)

Note: you may need to restart the kernel to use updated packages.
KeyError: authentication token for LMNR_PROJECT_API_KEY is undefined
Skipping Laminar.initialize()
